In [1]:
import requests
from functools import reduce
import pandas as pd

In [2]:
url = r"https://opendata.rijksoverheid.nl/v1/sources/rijksoverheid/infotypes/schoolholidays"

In [3]:
parameters = {"output" : "JSON"}
region = "noord"
# Note that after "2020 - 2021" there are no spaces between the years
selected_years = ["2019 - 2020", "2020 - 2021"]

In [4]:
r = requests.get(url, params=parameters)

In [5]:
years = filter(lambda x : x.get("content")[0].get("schoolyear").strip() in selected_years, r.json())

vacations = map(lambda x : x.get("content")[0].get("vacations"), years)

comp_regions = map(lambda y : [(x.get("compulsorydates"), filter(lambda z : z.get("region") == region or z.get("region") == "heel Nederland", x.get("regions"))) for x in y], vacations)


dates = map(lambda x : ([(compulsory == "true", [(date.get("startdate"), date.get("enddate")) for date in dates][0]) for compulsory,dates in x]), comp_regions)
dates3d = reduce(lambda x, y : x + y, dates)
dates2d = [(x,y,z) for x,(y,z) in dates3d]

df = pd.DataFrame(dates2d, columns=["Compulsory", "Start date", "End date"])
df.head()

,Compulsory,Start date,End date
0,False,2019-10-19T22:00:00.000Z,2019-10-27T21:59:00.000Z
1,True,2019-12-21T23:00:00.000Z,2020-01-05T22:59:00.000Z
2,False,2020-02-15T23:00:00.000Z,2020-02-23T22:59:00.000Z
3,True,2020-04-25T22:00:00.000Z,2020-05-03T21:59:00.000Z
4,True,2020-07-04T22:00:00.000Z,2020-08-16T21:59:00.000Z


In [6]:
df["Start date"] = pd.to_datetime(df["Start date"])
df["End date"] = pd.to_datetime(df["End date"])

In [7]:
iterator = df.iterrows()
i, row = next(iterator)
full_range = pd.date_range(start=row["Start date"], end=row["End date"]).to_series().append([pd.date_range(start=row["Start date"], end=row["End date"]).to_series() for i,row in iterator])

In [8]:
full_range = full_range.dt.date
full_range

2019-10-19 22:00:00+00:00    2019-10-19
2019-10-20 22:00:00+00:00    2019-10-20
2019-10-21 22:00:00+00:00    2019-10-21
2019-10-22 22:00:00+00:00    2019-10-22
2019-10-23 22:00:00+00:00    2019-10-23
                                ...    
2021-08-17 22:00:00+00:00    2021-08-17
2021-08-18 22:00:00+00:00    2021-08-18
2021-08-19 22:00:00+00:00    2021-08-19
2021-08-20 22:00:00+00:00    2021-08-20
2021-08-21 22:00:00+00:00    2021-08-21
Length: 164, dtype: object